# Integration mittels Monte-Carlo-Simulation

Erweiterung von Teil 1 um eine [Monte-Carlo-Simulation](https://mathepedia.de/Monte-Carlo-Methode.html). 

Hier soll der Bereich (Obere und Untere Grenze) der Simulation entweder manuell gewählt werden oder durch den Algorithmus bestimmt werden.